## 1. Configuração Inicial e Instalação de Dependências

In [1]:
!pip install numpy -q
!pip install python-dotenv -q
!pip install requests -q
!pip install opencv-python -q
!pip install opencv-python-headless -q
!pip install google-cloud-storage -q
!pip install google-cloud-bigquery -q
!pip install pandas-gbq google-cloud-bigquery pyarrow -q
!pip install pandas-gbq --upgrade -q


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\User

## 2. Importação de Bibliotecas

In [2]:
from google.cloud import bigquery
from google.cloud import storage
from dotenv import load_dotenv
import os
import cv2
import numpy as np
import pandas as pd
import base64
import requests
import json
import re
from datetime import datetime

## 3. Credenciais e Variáveis de Ambiente

In [ ]:
# Célula de Inicialização de Variáveis e Credenciais

# 1. Carrega todas as variáveis do arquivo .env para o ambiente
load_dotenv(r"C:\Users\mathe\Downloads\archive (10)\.env.txt")

# 2. Garante que o caminho da chave de serviço (ADC)

chave_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if chave_path:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = chave_path
    print("Caminho da chave de serviço configurado via os.environ.")
else:
    print("Atenção: GOOGLE_APPLICATION_CREDENTIALS não foi encontrado no .env!")


Caminho da chave de serviço configurado via os.environ.


In [4]:
# Puxar os valores das variáveis de ambiente
bucket_name = os.getenv("GCP_BUCKET_NAME")
project_id = os.getenv("GCP_PROJECT_ID")

# Configuracoes adicionais
prefix = ""
dataset_id = "processamento_imagens"
table_id = "dados-imagens-processadas"
x, y, w, h = 182, 322, 877, 1026

## 4. Definição de Funções

### 4.1. Google Cloud Storage (GCS)

In [5]:
def listar_imagens_bucket(bucket_name, prefix=""):
    """
    Listar os objetos (imagens) em um bucket do Google Cloud Storage.
    """
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return [blob.name for blob in blobs]

In [6]:

def baixar_imagem_bucket(bucket_name, imagem_nome):
    """
    Fazer o download de uma imagem do bucket do Google Cloud Storage.
    """
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(imagem_nome)
    return np.asarray(bytearray(blob.download_as_bytes()), dtype=np.uint8)

### 4.2. BigQuery (Controle de Processamento)

In [7]:
def verificar_imagem_processada(image_name):
    """
    Verificar se a imagem já foi processada consultando o BigQuery.
    """
    client = bigquery.Client(project=project_id)
    query = f"""
        SELECT COUNT(*) as count
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE image_name = @image_name
    """
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("image_name", "STRING", image_name)
        ]
    )
    query_job = client.query(query, job_config=job_config)
    results = query_job.result()
    for row in results:
        return row.count > 0
    return False

In [8]:

def registrar_imagem_processada(image_name):
    """
    Registrar uma imagem como processada no BigQuery.
    """
    client = bigquery.Client(project=project_id)
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)
    rows_to_insert = [
        {"image_name": image_name, "processed_date": datetime.utcnow().isoformat()}
    ]
    errors = client.insert_rows_json(table, rows_to_insert)
    if errors:
        raise RuntimeError(f"Erro ao inserir no BigQuery: {errors}")

### 4.3. Pré-processamento de Imagem

In [9]:
# Função para recortar a área de interesse
def extrair_etiqueta(img, x, y, w, h):
    """
    Recorta a região da etiqueta usando as coordenadas.
    """
    etiqueta = img[y:y + h, x:x + w]  
    return etiqueta

# Carregar uma nova imagem
imagem = cv2.imread('cropped_image.jpg')

if imagem is not None:
    # Recortar a etiqueta
    etiqueta = extrair_etiqueta(imagem, x, y, w, h)

    # Mostrar e salvar o recorte
    cv2.imwrite("etiqueta_recortada.jpg", etiqueta)
else:
    print("Erro: Não foi possível carregar a imagem. Verifique o caminho ou o arquivo.")

### 4.4. OCR (Google Vision API)

In [10]:
def ocrGoogleVisionApi(imagem):
    
    # 1. PEGAR A CHAVE DA API DA VARIÁVEL DE AMBIENTE
    api_key = os.getenv("GOOGLE_VISION_API_KEY")

    if not api_key:
        return "Erro: A chave 'GOOGLE_VISION_API_KEY' não foi encontrada nas variáveis de ambiente."

    # Codificar a imagem em base64
    _, buffer = cv2.imencode('.jpg', imagem)
    my_base64 = base64.b64encode(buffer).decode("utf-8") 

    # 2. USAR A CHAVE NA URL
    # URL da API com a chave de API
    url = f'https://vision.googleapis.com/v1/images:annotate?key={api_key}'

    # Dados do pedido
    data = {
        "requests": [
            {
                "image": {
                    "content": my_base64
                },
                "features": [
                    {"type": "TEXT_DETECTION"}
                ],
            }
        ]
    }

    # Enviar o pedido POST
    r = requests.post(url=url, data=json.dumps(data), headers={"Content-Type": "application/json"})

    # Verificar se houve erro
    if r.status_code != 200:
        return "Erro: " + r.text

    # Extrair as anotações de texto
    texts = r.json()['responses'][0].get('textAnnotations', [])

    # Coletar os resultados
    results = [t['description'] for t in texts]
    return results

### 4.5. Tratamento de Texto

In [11]:
#Tratamento dos dados

# Pré-compila todos os padrões
PADRAO_VALIDADE = re.compile(
    r'(?:Val(?:\.|idade)?[:\s]*)'                
    r'(?P<validade>\d{2}/\d{4}|\d{2}/\d{2}/\d{4})',
    re.IGNORECASE
)
PADRAO_CODIGO = re.compile(r'Cod[:\s]*(?P<codigo>[\w\.-]+)', re.IGNORECASE)
PADRAO_LOTE   = re.compile(r'Lote[:\s]*(?P<lote>\d+)',             re.IGNORECASE)
PADRAO_VOLUME = re.compile(r'Cont[.\s:]*?(?P<volume>\d+(?:L|mL))', re.IGNORECASE)

def extrair_dados(texto_fragmentos):
    # 1) Monta string única, limpa quebras de linha
    texto = ' '.join(texto_fragmentos).replace('\n', ' ').replace('\r', ' ')

    resultados = {
        'validade': None,
        'codigo_produto': None,
        'lote': None,
        'volume': None,
    }

    # 2) Busca validade e valida
    m = PADRAO_VALIDADE.search(texto)
    if m:
        val = m.group('validade')
        # se veio no formato MM/YYYY, assume dia = 01
        to_parse = val if val.count('/') == 2 else f"01/{val}"
        try:
            dt = datetime.strptime(to_parse, '%d/%m/%Y')
            resultados['validade'] = dt.strftime('%d/%m/%Y')
        except ValueError:
            pass

    # 3) Código do produto
    m = PADRAO_CODIGO.search(texto)
    if m:
        resultados['codigo_produto'] = m.group('codigo')

    # 4) Lote (só aceita dígitos)
    m = PADRAO_LOTE.search(texto)
    if m and int(m.group('lote')) > 0:
        resultados['lote'] = m.group('lote')

    # 5) Volume (ex.: 18L ou 100mL)
    m = PADRAO_VOLUME.search(texto)
    if m:
        vol = m.group('volume')
        num = int(re.match(r'\d+', vol).group())
        if num > 0:
            resultados['volume'] = vol

    return resultados

## 5. Execução do Processamento Principal

In [12]:
dados_extraidos = []

# Processo principal
try:
    imagens = listar_imagens_bucket(bucket_name, prefix)
    for nova_imagem in imagens:
        if not verificar_imagem_processada(nova_imagem):
            print(f"Processando nova imagem: {nova_imagem}")

            imagem_bytes = baixar_imagem_bucket(bucket_name, nova_imagem)
            imagem_original = cv2.imdecode(np.frombuffer(imagem_bytes, dtype=np.uint8), cv2.IMREAD_COLOR)
            if imagem_original is None:
                raise ValueError("Erro ao decodificar a imagem baixada.")

            etiqueta_recortada = extrair_etiqueta(imagem_original, x, y, w, h)
            texto_extraido = ocrGoogleVisionApi(etiqueta_recortada)
            print("Texto extraído:", texto_extraido)

            #  Tratar o texto sujo com regex logo após o OCR
            dados = extrair_dados(texto_extraido)
            print(" Dados extraídos:", dados)

            dados_extraidos.append({
                'imagem': nova_imagem,
                'validade': dados['validade'],
                'codigo_produto': dados['codigo_produto'],
                'lote': dados['lote'],
                'volume': dados['volume']
            })

            registrar_imagem_processada(nova_imagem)
        else:
            print(f"Imagem já processada: {nova_imagem}")
except Exception as e:
    print(f"Ocorreu um erro ao processar a imagem: {str(e)}")


Imagem já processada: cropped_2.jpeg
Imagem já processada: cropped_image.jpg
Imagem já processada: tinta_luxens.JPG


## 6. Insert de Dados no BigQuery

### 6.1. Criação do DataFrame

In [27]:
df = pd.DataFrame(dados_extraidos)
df

NameError: name 'pd' is not defined

### 6.2. Gravação no BigQuery

In [13]:
# Gravar dados na tabela do big query


# ID do projeto
project_id = 'challenge-421602'

# Tabela de métricas
to_gbq(
    df,
    destination_table='processamento_imagens.texto_extraido',
    project_id=project_id,
    if_exists='append' 
)

NameError: name 'to_gbq' is not defined